#  Diagnostic de maladies par symptômes (NLP + Naive Bayes)


Ce notebook montre comment utiliser **spaCy** et **Multinomial Naive Bayes** pour diagnostiquer une maladie à partir d'une description textuelle des **symptômes**.


Importation des dépendances 

In [8]:
import spacy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

nlp = spacy.load("fr_core_news_sm")
print("spaCy chargé avec succès !")

spaCy chargé avec succès !


Importation du dataset dans un dataframe Pandas

In [ ]:
df = pd.read_csv("dataset.csv")

df

,symptomes,maladie
0,"maux de tête, toux sèche, fièvre, frissons",Grippe
1,"toux sèche, difficultés respiratoires, douleur...",Covid-19
2,"douleurs musculaires, transpiration abondante,...",Paludisme
3,"nausée, fatigue, maux d’estomac, perte d’appét...",Gastro-entérite
4,"maux de gorge, maux de tête, nez qui coule",Rhume
...,...,...
1195,"fièvre, ganglions enflés, gorge très douloureu...",Angine
1196,"fièvre, fatigue, douleurs musculaires, perte d...",Covid-19
1197,"fièvre élevée, gorge très douloureuse, ganglio...",Angine
1198,"fièvre, amygdales gonflées, difficulté à avale...",Angine


Fonction de prétraitement utilisant spaCy 

In [9]:
def preprocess_text(text):
    """
    Nettoie le texte avec spaCy.
    """
    doc = nlp(text.lower())
    tokens = [
        token.lemma_ for token in doc 
        if not token.is_stop and not token.is_punct and not token.is_space
    ]
    return " ".join(tokens)

Prétaitement et données et séparation du dataset en données d'entrainement et de teste

In [12]:
df['symptome_clean'] = df['symptomes'].apply(preprocess_text)
df[['symptomes', 'symptome_clean', 'maladie']]


X = df['symptome_clean']
Y = df['maladie']


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

df

,symptomes,maladie,symptome_clean
0,"maux de tête, toux sèche, fièvre, frissons",Grippe,mal tête toux sec fièvre frisson
1,"toux sèche, difficultés respiratoires, douleur...",Covid-19,toux sec difficulté respiratoire douleur muscu...
2,"douleurs musculaires, transpiration abondante,...",Paludisme,douleur musculaire transpiration abondant fris...
3,"nausée, fatigue, maux d’estomac, perte d’appét...",Gastro-entérite,nauser fatigue mal estomac perte appétit diarrhée
4,"maux de gorge, maux de tête, nez qui coule",Rhume,mal gorge mal tête nez coule
...,...,...,...
1195,"fièvre, ganglions enflés, gorge très douloureu...",Angine,fièvre ganglion enfler gorge douloureux fièvre...
1196,"fièvre, fatigue, douleurs musculaires, perte d...",Covid-19,fièvre fatigue douleur musculaire perte goût
1197,"fièvre élevée, gorge très douloureuse, ganglio...",Angine,fièvre élevé gorge douloureux ganglion enfler ...
1198,"fièvre, amygdales gonflées, difficulté à avale...",Angine,fièvre amygdale gonflé difficulter avaler fièv...


Vectorisation et Entrainement du modèle

In [13]:
# Pipeline complet
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(max_features=1000)),
    ('classifier', MultinomialNB(alpha=1.0))
])

# Entraînement
pipeline.fit(X_train, Y_train)
print("Modèle entraîné !")

Modèle entraîné !


Eval du modèle

In [17]:
# Prédictions
y_pred = pipeline.predict(X_test)

# Métriques
print(f"Précision : {accuracy_score(Y_test, y_pred):.2f}")
print("\nRapport de classification :")
print(classification_report(Y_test, y_pred))

# Matrice de confusion
# cm = confusion_matrix(Y_test, y_pred, labels=['positif', 'négatif'])
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
#             xticklabels=['positif', 'négatif'], 
#             yticklabels=['positif', 'négatif'])
# plt.xlabel('Prédit')
# plt.ylabel('Réel')
# plt.title('Matrice de confusion')
# plt.show()

Précision : 0.98

Rapport de classification :
                 precision    recall  f1-score   support

         Angine       1.00      1.00      1.00        41
       Covid-19       1.00      0.94      0.97        35
Gastro-entérite       1.00      1.00      1.00        41
         Grippe       0.95      0.95      0.95        41
      Paludisme       1.00      1.00      1.00        43
          Rhume       0.95      1.00      0.97        39

       accuracy                           0.98       240
      macro avg       0.98      0.98      0.98       240
   weighted avg       0.98      0.98      0.98       240



In [18]:
nouveaux_symtomes = [
    "Nez bouché, mal de tête, Angine.",
    "Mal de ventre, diarrhée, nausée.",
    "Toux sèche, fièvre, fatigue intense."
]

for phrase in nouveaux_symtomes:
    clean = preprocess_text(phrase)
    pred = pipeline.predict([clean])[0]
    prob = pipeline.predict_proba([clean])[0]
    confidence = max(prob)
    print(f"Phrase : '{phrase}' → {pred.upper()} (confiance: {confidence:.2f})")

Phrase : 'Nez bouché, mal de tête, Angine.' → GRIPPE (confiance: 0.50)
Phrase : 'Mal de ventre, diarrhée, nausée.' → GASTRO-ENTÉRITE (confiance: 0.98)
Phrase : 'Toux sèche, fièvre, fatigue intense.' → GRIPPE (confiance: 0.53)


In [19]:
# Sauvegarde (vectorizer + modèle)
joblib.dump(pipeline, 'modele_nlp_diagnostic.pkl')
print("Modèle sauvegardé : modele_nlp_diagnostic.pkl")

# Sauvegarde aussi la fonction de prétraitement (optionnel, pour réutilisation)
joblib.dump(preprocess_text, 'preprocess_func.pkl')
print("Fonction de prétraitement sauvegardée.")

Modèle sauvegardé : modele_nlp_diagnostic.pkl
Fonction de prétraitement sauvegardée.
